# VGG16 fine-tuning for defect detection

This section builds a VGG16 model with ImageNet weights, lets you choose any valid input shape (HxWx3), and fine-tune only the last layers you specify.

In [1]:
import os
import sys

import numpy as np
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))
from SRModels.defect_detection_models.VGG16_model import FineTunedVGG16
from SRModels.loading_methods import load_defects_dataset_as_patches
from SRModels.constants import VGG_PATCH_SIZE, VGG_STRIDE, RANDOM_SEED

In [2]:
HR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/HR"))
CLASS_LABELS_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/class_labels_map.pkl"))

In [3]:
# X ->  High-resolution patches (model input)
# y -> Class labels (target)
X, y = load_defects_dataset_as_patches(HR_ROOT, patch_size=VGG_PATCH_SIZE, stride=VGG_STRIDE, class_map_path=CLASS_LABELS_PATH)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)

In [4]:
# how many samples per class
(unique, counts) = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

{0: 11456, 1: 8576}


In [5]:
print(f"X shape: {X.shape}, Y shape: {y.shape}")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

unique, counts = np.unique(y, return_counts=True)
print(f"Class distribution: {dict(zip(unique, counts))}")

X shape: (20032, 96, 96, 3), Y shape: (20032,)
X_train shape: (14422, 96, 96, 3), y_train shape: (14422,)
X_val shape: (1603, 96, 96, 3), y_val shape: (1603,)
X_test shape: (4007, 96, 96, 3), y_test shape: (4007,)
Class distribution: {0: 11456, 1: 8576}


In [6]:
model = FineTunedVGG16()

model.setup_model(
    input_shape=X.shape[1:],
    num_classes=np.unique(y).shape[0],
    train_last_n_layers=6,
    base_trainable=True,
    dropout_rate=0.3,
    l2_reg=1e-4,
    learning_rate=1e-3,
    loss="sparse_categorical_crossentropy",
    from_pretrained=False,
    pretrained_path=None
)

Model: "vgg16_finetune"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 gap (GlobalAveragePooling2D  (None, 512)              0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                    

In [7]:
history = model.fit(
    X_train, y_train,
    X_val, y_val,
    batch_size=32,
    epochs=1,
    use_augmentation=True
)

451/451 [==============================] - 26s 46ms/step - loss: 0.3913 - accuracy: 0.8310 - val_loss: 0.2440 - val_accuracy: 0.9239 - lr: 0.0010


In [8]:
model.evaluate(X_test, y_test)

# Evaluate and prepare metrics dictionary
results = model.evaluate(X_test, y_test)

# Extract last epoch train/val metrics from history
train_loss = float(history.history['loss'][-1]) if 'loss' in history.history else None
val_loss = float(history.history['val_loss'][-1]) if 'val_loss' in history.history else None
train_accuracy = float(history.history['accuracy'][-1]) if 'accuracy' in history.history else None
val_accuracy = float(history.history['val_accuracy'][-1]) if 'val_accuracy' in history.history else None

metrics_dict = {
    "eval_loss": float(results[0]),
    "eval_accuracy": float(results[1]),
    "final_train_loss": train_loss,
    "final_val_loss": val_loss,
    "final_train_accuracy": train_accuracy,
    "final_val_accuracy": val_accuracy,
}

126/126 [==============================] - 3s 23ms/step - loss: 0.2799 - accuracy: 0.8984
Loss: 0.2799, Accuracy: 0.8984
126/126 [==============================] - 3s 20ms/step - loss: 0.2799 - accuracy: 0.8984
Loss: 0.2799, Accuracy: 0.8984


In [9]:
model.save()

Model saved to models/VGG16\VGG16_20250909_173922.h5
